In [2]:
import javalang
import re
import os
import sys
import json


修改folder为Training, Eval. 在jsonInput4JavaParser下面得到json文件

这个json文件
1. key是行号, value是一个字典
2. 包含了可以被java parser处理的compilation unit
3. 把从sourceRoot输入输出的部分改成读写字符串. 

In [37]:
folder = "Testing"

input_folder = os.path.join('../input/', folder)
out_fp = os.path.join('../jsonInput4JavaParser/', folder+'.json')

assert_file = 'assertLines.txt'
testMethod_file = 'testMethods.txt'

assertion_fp = os.path.join(input_folder,assert_file)
testMethod_fp = os.path.join(input_folder,testMethod_file)

split_re = re.compile("\"<AssertPlaceHolder>\" ;[ }]*")


assertLines.txt

```java
assertEquals(a, b)

class X{ void var24678() { assertEquals(a, b); } }
```

testMethods.txt
```java
test_method(){...} focal_method(Type, Type){...}

class Y {
    void test_method(){
        /* AssertPlaceHolder 变成注释 */
    }
}

class Z{
    void focal_method(Type a, Type b){
        ...
    }
}
```

In [42]:
with open(assertion_fp,'r',encoding='utf-8') as f:
    assertLines = f.readlines()

with open(testMethod_fp,'r',encoding='utf-8') as f:
    testMethods = f.readlines()

In [43]:
outputs = {}

In [44]:
class ParseError(Exception):
    pass

In [45]:
def process_assertLine(assertLine):
    return 'class X { void var24678() { ' + assertLine + '; } }'

In [49]:
def split_testmethod(testMethod):
    match = split_re.search(testMethod)
    if not match:
        raise ParseError('cannot find test method and focal method')
    idx = match.span()[1]
    tm = testMethod[0:idx]
    fm = testMethod[idx:]
    return tm, fm

def split_func(func):
    lp = func.find('(')
    rp = func.find(')')
    return func[:lp+1], func[lp+1:rp], func[rp:]

def add_param_def(str):
    if str.strip() == '':
        return str
    type_definition = str.split(',')
    res = ','.join([x+" var24678" for x in type_definition])
    return res

def deal_with_atlas_fm(fm):
    func_name, params, body = split_func(fm)
    params = add_param_def(params)
    return func_name+params+body

def simp_fqdm(assertion, testMethod):
    tm, fm = split_testmethod(testMethod)
    tm = 'void ' + tm.replace('"<AssertPlaceHolder>" ;', '/* "<AssertPlaceHolder>" ; */')
    fm = 'void ' + fm
    fm = deal_with_atlas_fm(fm)
    sp = '\n/* separator */\n'
    assertion = 'void var24678(){' + assertion + ';'+'}'
    return 'class X{\n' + assertion + '}\n',  'class Y{\n' + tm +'}\n',  'class Z{\n'+fm + '}\n'


In [50]:
error_idx = []
for idx,(assertLine,testMethod) in enumerate(zip(assertLines,testMethods)):
    try:
        a,t,f = simp_fqdm(assertLine,testMethod)
    except ParseError as e:
        error_idx.append(idx)
        continue

    case = {}
    case['assertion'] = a
    case['testMethod'] = t
    case['focalMethod'] = f

    outputs[idx] = case
    

In [51]:
with open(out_fp,'w',encoding='utf-8') as f:
    json.dump(outputs,f,indent=4)


In [52]:
print(error_idx)

[135, 494, 562, 1289, 1589, 1688, 1832, 2125, 2311, 2627, 2716, 2769, 2801, 3133, 3422, 3472, 5951, 6303, 6330, 6764, 7700, 7883, 8003, 8884, 9816, 10323, 10568, 11148, 11502, 12175, 12296, 12577, 12589, 12647, 12908, 13032, 13515, 13712, 14444, 14653, 15015, 15067, 15169, 15434, 15800, 15905, 17168, 17193, 17611, 17990, 18189, 18433, 18449]
